In [1]:
# =================================
# v3.0 >> 업데이트 부분 자동화


In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
save_dir='C:/Yeajin/RealData/마일스톤/crawling/'
chrome_dir='C:/Yeajin/chromedriver_win32/chromedriver'

In [15]:
apply_df=pd.read_csv('청구종목.csv',encoding='utf-8',index_col='Unnamed: 0')
approval_df=pd.read_csv('승인종목.csv',encoding='utf-8',index_col='Unnamed: 0')
ir_df=pd.read_csv('기업 IR 일정.csv',encoding='utf-8',index_col='Unnamed: 0')
presche_df=pd.read_csv('수요예측일정.csv',encoding='utf-8',index_col='Unnamed: 0') 
result_df=pd.read_csv('수요예측결과.csv',encoding='utf-8',index_col='Unnamed: 0')
subs_df=pd.read_csv('공모청약일정.csv',encoding='utf-8',index_col='Unnamed: 0') 
newlisting_df=pd.read_csv('신규상장.csv',encoding='utf-8',index_col='Unnamed: 0')

In [16]:
def integrated_table1(apply_df,approval_df):
    approval_df[['청구일','기업명','승인일','자본금(백만)','매출액(백만)',
                        '당기순이익(백만)','주간사','주업종']]
    
    applylist=[]
    for i in approval_df.index:
        apply_year=approval_df['승인일'][i][:4]
        apply=approval_df['청구일'][i]
        applydate=str(apply_year)+'/'+str(apply)
        applylist.append(applydate)

    approval_df['청구일']=applylist    
    apply_df=apply_df.rename({'상태':'승인일'},axis=1)
    result=pd.concat([apply_df,approval_df])
    
    return result
    

In [17]:
def integrated_table2(apply_df,approval_df,ir_df):
    result=integrated_table1(apply_df,approval_df)
    result=pd.merge(result,ir_df,on='기업명',how='outer')
    result['기업명']=result['기업명'].str.strip()
    
    companys=[]
    result['주간사_x']=result['주간사_x'].apply(str)
    result['주간사_y']=result['주간사_y'].apply(str)
    for i in result.index:
        xcompany=result['주간사_x'][i]
        ycompany=result['주간사_y'][i]
        if xcompany=='nan' and ycompany!='nan':
            company=ycompany
            companys.append(company)
        elif ycompany=='nan' and xcompany!='nan':
            company=xcompany
            companys.append(company)
        elif ycompany=='nan' and xcompany=='nan':
            company=np.nan
            companys.append(company)
        elif ycompany==xcompany:
            company=xcompany
            companys.append(company)
        else:
            company=str(xcompany)+','+str(ycompany)
            companys.append(company)
#             print(i,result['기업명'][i],company)
            
    result.drop(['주간사_x','주간사_y'],axis=1,inplace=True)
    result['주간사']=companys
    
    return result

In [18]:
def integrated_table3(apply_df,approval_df,ir_df,presche_df):
    result=integrated_table2(apply_df,approval_df,ir_df)
    presche_df['기업명']=presche_df['기업명'].str.strip()
    presche_df['희망공모가(원)']=presche_df['희망공모가(원)'].str.strip()
    result=pd.merge(result,presche_df,on='기업명',how='outer')
    
    companys=[]
    result['주간사_x']=result['주간사_x'].apply(str)
    result['주간사_y']=result['주간사_y'].apply(str)
    for i in result.index:
        xcompany=result['주간사_x'][i]
        ycompany=result['주간사_y'][i]
        if xcompany=='nan' and ycompany!='nan':
            company=ycompany
            companys.append(company)
        elif ycompany=='nan' and xcompany!='nan':
            company=xcompany
            companys.append(company)
        elif ycompany=='nan' and xcompany=='nan':
            company=np.nan
            companys.append(company)
        elif ycompany==xcompany:
            company=xcompany
            companys.append(company)
        else:
            company=ycompany
            companys.append(company)
#             print(i,result['기업명'][i],company)
        
    result.drop(['주간사_x','주간사_y'],axis=1,inplace=True)
    result['주간사']=companys
    
    desired_prices=[]
    result['공모희망가']=result['공모희망가'].apply(str)
    result['희망공모가(원)']=result['희망공모가(원)'].apply(str)
    for i in result.index:
        x=result['공모희망가'][i]
        y=result['희망공모가(원)'][i]
        if x=='nan' and y!='nan':
            value=y
            desired_prices.append(value)
        elif y=='nan' and x!='nan':
            value=x
            desired_prices.append(value)
        elif y=='nan' and x=='nan':
            value=np.nan
            desired_prices.append(value)
        elif y==x:
            value=x
            desired_prices.append(value)
        else:
            value=str(x)+','+str(y)
            desired_prices.append(value)
#             print(i,result['기업명'][i],value)
            
    result.drop(['공모희망가','희망공모가(원)'],axis=1,inplace=True)
    result['희망공모가(원)']=desired_prices
    
    return result


In [19]:
def integrated_table4(apply_df,approval_df,ir_df,presche_df,result_df):
    result3=integrated_table3(apply_df,approval_df,ir_df,presche_df)
    result_df['기업명']=result_df['기업명'].str.strip()
    result=pd.merge(result3,result_df,on='기업명',how='outer')
    
    companys=[]
    result['주간사_x']=result['주간사_x'].apply(str)
    result['주간사_y']=result['주간사_y'].apply(str)
    for i in result.index:
        xcompany=result['주간사_x'][i]
        ycompany=result['주간사_y'][i]
        if xcompany=='nan' and ycompany!='nan':
            company=ycompany
            companys.append(company)
        elif ycompany=='nan' and xcompany!='nan':
            company=xcompany
            companys.append(company)
        elif ycompany=='nan' and xcompany=='nan':
            company=np.nan
            companys.append(company)
        elif ycompany==xcompany:
            company=xcompany
            companys.append(company)
        else:
            company=str(xcompany)+str(ycompany)
            companys.append(company)
#             print(i,result['기업명'][i],company)
            
    result.drop(['주간사_x','주간사_y'],axis=1,inplace=True)
    result['주간사']=companys
    
    
    confirmed_prices=[]
    result['확정공모가']=result['확정공모가'].apply(str)
    result['공모가(원)']=result['공모가(원)'].apply(str)
    for i in result.index:
        x=result['확정공모가'][i]
        y=result['공모가(원)'][i]
        if x=='nan' and y!='nan':
            value=y
            confirmed_prices.append(value)
        elif y=='nan' and x!='nan':
            value=x
            confirmed_prices.append(value)
        elif y=='nan' and x=='nan':
            value=np.nan
            confirmed_prices.append(value)
        elif y==x:
            value=x
            confirmed_prices.append(value)
        else:
            value=str(x)+','+str(y)
            confirmed_prices.append(value)
#             print(i,result['기업명'][i],value)

    result.drop(['확정공모가','공모가(원)'],axis=1,inplace=True)
    result['확정공모가']=confirmed_prices
    
    
    prices=[]
    result['공모금액(백만)']=result['공모금액(백만)'].apply(str)
    result['공모금액(백만원)']=result['공모금액(백만원)'].apply(str)
    for i in result.index:
        x=result['공모금액(백만)'][i]
        y=result['공모금액(백만원)'][i]
        if x=='nan' and y!='nan':
            value=y
            prices.append(value)
        elif y=='nan' and x!='nan':
            value=x
            prices.append(value)
        elif y=='nan' and x=='nan':
            value=np.nan
            prices.append(value)
        elif y==x:
            value=x
            prices.append(value)
        else:
            value=ycompany
            prices.append(value)
#             print(i,result['기업명'][i],value)

    
    result.drop(['공모금액(백만)','공모금액(백만원)'],axis=1,inplace=True)
    result['공모금액(백만원)']=prices
    
    result['공모희망가(원)']=result['공모희망가(원)'].str.strip()
    desired_prices=[]
    result['공모희망가(원)']=result['공모희망가(원)'].apply(str)
    result['희망공모가(원)']=result['희망공모가(원)'].apply(str)
    for i in result.index:
        x=result['공모희망가(원)'][i]
        y=result['희망공모가(원)'][i]
        if x=='nan' and y!='nan':
            value=y
            desired_prices.append(value)
        elif y=='nan' and x!='nan':
            value=x
            desired_prices.append(value)
        elif y=='nan' and x=='nan':
            value=np.nan
            desired_prices.append(value)
        elif x==y:
            value=x
            desired_prices.append(value)
        else:
            value=y
            desired_prices.append(value)
#             print(i,result['기업명'][i],value)

    result.drop(['공모희망가(원)'],axis=1,inplace=True)
    result['희망공모가(원)']=desired_prices

    demand_schedule=[]
    result['수요예측일']=result['수요예측일'].apply(str)
    result['예측일']=result['예측일'].apply(str)
    for i in result.index:
        x=result['수요예측일'][i]
        y=result['예측일'][i]
        if x=='nan' and y!='nan':
            value=y
            demand_schedule.append(value)
        elif y=='nan' and x!='nan':
            value=x
            demand_schedule.append(value)
        elif y=='nan' and x=='nan':
            value=np.nan
            demand_schedule.append(value)
        elif x==y:
            value=y
            demand_schedule.append(value)
        else:
            value=y
            demand_schedule.append(value)
#             print(i,result['기업명'][i],value)

    result.drop(['예측일'],axis=1,inplace=True)
    result['수요예측일']=demand_schedule

    return result

In [20]:
def integrated_table5(apply_df,approval_df,ir_df,presche_df,result_df,subs_df):
    result4=integrated_table4(apply_df,approval_df,ir_df,presche_df,result_df)
    subs_df['기업명']=subs_df['기업명'].str.strip()
    result=pd.merge(result4,subs_df,on='기업명',how='outer')
    result['희망공모가(원)']=result['희망공모가(원)'].str.strip()
    
    desired_prices=[]
    result['희망공모가(원)']=result['희망공모가(원)'].apply(str)
    result['희망공모가']=result['희망공모가'].apply(str)
    for i in result.index:
        x=result['희망공모가(원)'][i]
        y=result['희망공모가'][i]
        if x=='nan' and y!='nan':
            value=y
            desired_prices.append(value)
        elif y=='nan' and x!='nan':
            value=x
            desired_prices.append(value)
        elif y=='nan' and x=='nan':
            value=np.nan
            desired_prices.append(value)
        elif x==y:
            value=x
            desired_prices.append(value)
        else:
            value=x
            desired_prices.append(value)
#             print(i,result['기업명'][i],value)

    result.drop(['희망공모가'],axis=1,inplace=True)
    result['희망공모가(원)']=desired_prices
    
    confirmed_prices=[]
    result['확정공모가_x']=result['확정공모가_x'].apply(str)
    result['확정공모가_y']=result['확정공모가_y'].apply(str)
    for i in result.index:
        x=result['확정공모가_x'][i]
        y=result['확정공모가_y'][i]
        if x=='nan' and y!='nan':
            value=y
            confirmed_prices.append(value)
        elif y=='nan' and x!='nan':
            value=x
            confirmed_prices.append(value)
        elif y=='nan' and x=='nan':
            value=np.nan
            confirmed_prices.append(value)
        elif x==y:
            value=x
            confirmed_prices.append(value)
        else:
            value=x
            confirmed_prices.append(value)
#             print(i,result['기업명'][i],value)

    result.drop(['확정공모가_x','확정공모가_y'],axis=1,inplace=True)
    result['확정공모가']=confirmed_prices
    
    companys=[]
    result['주간사_x']=result['주간사_x'].apply(str)
    result['주간사_y']=result['주간사_y'].apply(str)
    for i in result.index:
        xcompany=result['주간사_x'][i]
        ycompany=result['주간사_y'][i]
        if xcompany=='nan' and ycompany!='nan':
            company=ycompany
            companys.append(company)
        elif ycompany=='nan' and xcompany!='nan':
            company=xcompany
            companys.append(company)
        elif ycompany=='nan' and xcompany=='nan':
            company=np.nan
            companys.append(company)
        elif ycompany==xcompany:
            company=xcompany
            companys.append(company)
        else:
            company=str(xcompany)+str(ycompany)
            companys.append(company)
#             print(i,result['기업명'][i],company)

    result.drop(['주간사_x','주간사_y'],axis=1,inplace=True)
    result['주간사']=companys
    
    return result


In [21]:
def integrated_table6(apply_df,approval_df,ir_df,presche_df,result_df,subs_df,newlisting_df):
    result5=integrated_table5(apply_df,approval_df,ir_df,presche_df,result_df,subs_df)
    newlisting_df['기업명']=newlisting_df['기업명'].str.strip()
    result=pd.merge(result5,newlisting_df,on='기업명',how='outer')
    result['공모가(원)']=result['공모가(원)'].str.strip()
    
    prices=[]
    result['확정공모가']=result['확정공모가'].apply(str)
    result['공모가(원)']=result['공모가(원)'].apply(str)
    for i in result.index:
        x=result['확정공모가'][i]
        y=result['공모가(원)'][i]
        if x=='nan' and y!='nan':
            value=y
            prices.append(value)
        elif y=='nan' and x!='nan':
            value=x
            prices.append(value)
        elif y=='nan' and x=='nan':
            value=np.nan
            prices.append(value)
        elif x==y:
            value=x
            prices.append(value)
        else:
            value=str(x)+','+str(y)
            prices.append(value)
            
    result.drop(['확정공모가','공모가(원)'],axis=1,inplace=True)
    result['확정공모가(원)']=prices

    return result

In [22]:
def integrated_complete(apply_df,approval_df,ir_df,presche_df,result_df,subs_df,newlisting_df):
    result6=integrated_table6(apply_df,approval_df,ir_df,presche_df,result_df,subs_df,newlisting_df)
    result=result6[['기업명','청구일','승인일','자본금(백만)','매출액(백만)','당기순이익(백만)',
                '주업종','주간사','IR 일자','IR 장소/시간','수요예측일','희망공모가(원)','확정공모가(원)','기관경쟁률',
                '의무보유확약','공모금액(백만원)','공모주일정','청약경쟁률','분석링크',
                '신규상장일','현재가(원)','전일비(%)','공모가대비등락률(%)','시초가(원)',
                 '시초/공모(%)','첫날종가(원)']]

#     desired=result['희망공모가(원)'].tolist()
# #     print(len(desired))
# #     print(len(result['기업명'].tolist()))
#     low=[]
#     high=[]
#     for price in desired:
#         price=str(price).strip().split('~')
# #         print(price)
#         if len(price)==1:
#             low.append(np.nan)
#             high.append(np.nan)
#         else:
#             low.append(price[0])
#             high.append(price[1])
# #     print(len(low),len(high))
    
#     result.insert(11,'희망공모가(하단)',low)
#     result.insert(12,'희망공모가(상단)',high)

    d=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
    print(d,'기준으로 테이블이 완성되었습니다.')
    d=time.strftime('%y%m%d',time.localtime(time.time()))
    cr='수집'
    result.to_csv(f'기업 IPO 일정_{cr}.csv',encoding='utf-8')              # 주피터 내부 저장용
    result.to_csv(save_dir+f'기업 IPO 일정_{d}_{cr}.csv',encoding='utf-8')          # 로컬 디렉토리 저장용
    
    return result

In [24]:
integrated_complete(apply_df,approval_df,ir_df,presche_df,result_df,subs_df,newlisting_df).head(100)

2020-10-21 16:27:33 기준으로 테이블이 완성되었습니다.


,기업명,청구일,승인일,자본금(백만),매출액(백만),당기순이익(백만),주업종,주간사,IR 일자,IR 장소/시간,...,공모주일정,청약경쟁률,분석링크,신규상장일,현재가(원),전일비(%),공모가대비등락률(%),시초가(원),시초/공모(%),첫날종가(원)
0,이삭엔지니어링,2020/10/20,NaN,"3,062","32,759","2,679","컴퓨터 프로그래밍, 시스템 통합",신한금융투자,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,아모센스,2020/10/19,NaN,"4,447","62,028","-1,196",전자부품 제조업,신한금융투자,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,유안타스팩7호,2020/10/16,NaN,115,0,0,금융 지원 서비스업,유안타증권,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,유안타스팩8호,2020/10/16,NaN,24,0,0,금융 지원 서비스업,유안타증권,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,디비금융스팩9호,2020/10/15,NaN,84,0,0,기타 금융업,DB금융투자,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,아이디피,NaN,NaN,NaN,NaN,NaN,NaN,키움증권,2020/08/06,기관: 온라인-한국IR협의회 홈페이지 PM 03:10\n일반: 온라인-한국IR협의회...,...,2020.08.11~08.12,1149.83:1,http://www.38.co.kr/html/fund/index.htm?o=v&no...,2020/08/24,"8,220",0.12%,-16.12%,"13,200",34.69%,"12,500"
96,셀레믹스,NaN,NaN,NaN,NaN,NaN,NaN,대신증권,2020/08/03,기관: 온라인-유튜브 생중계 AM 10:00\n일반: 온라인-한국IR협의회 홈페이지...,...,2020.08.10~08.11,1176.45:1,http://www.38.co.kr/html/fund/index.htm?o=v&no...,2020/08/21,"21,250",1.19%,6.25%,"24,900",24.5%,"25,800"
97,미투젠,NaN,NaN,NaN,NaN,NaN,NaN,미래에셋대우,2020/07/30,기관: 온라인-유튜브 생중계 AM 10:30\n일반: 온라인-한국IR협의회 홈페이지...,...,2020.08.05~08.06,1010.86:1,http://www.38.co.kr/html/fund/index.htm?o=v&no...,2020/08/18,"27,250",0.74%,0.93%,"29,100",7.78%,"25,500"
98,브랜드엑스코퍼레이션,NaN,NaN,NaN,NaN,NaN,NaN,"대신증권,삼성증권",2020/07/30,기관: 63빌딩 4층 라벤더로즈마리홀 AM 11:30\n일반: 온라인-한국IR협의회...,...,2020.08.04~08.05,8.54:1,http://www.38.co.kr/html/fund/index.htm?o=v&no...,2020/08/13,"12,450",-0.40%,-4.23%,"12,900",-0.77%,"11,700"


In [34]:
# 이전 데이터에서 update
def df_update():
    ipo_before=pd.read_csv(f'기업 IPO 일정_업데이트.csv',encoding='utf-8')
    ipo_after=pd.read_csv(f'기업 IPO 일정_수집.csv',index_col='Unnamed: 0')
    
    ipo_before.set_index('기업명',inplace=True)
    ipo_after.set_index('기업명',inplace=True)

    #ipo_before.update(ipo_after,join='left',overwrite='True',filter_func=None,errors='ignore')

    ipo_before.update(ipo_after)
    
    ipo_before.to_csv(f'기업 IPO 일정_업데이트_before_add.csv',encoding='utf-8')            # 주피터 내부 저장용
    

    up='업데이트'
#     ipo_before.to_csv(f'기업 IPO 일정_업데이트.csv',encoding='utf-8')                 # 주피터 내부 저장용              
#    ipo_before.to_csv(save_dir+f'기업 IPO 일정_{d}_{up}.csv',encoding='utf-8')          # 로컬 디렉토리 저장용

In [27]:
df_update()

In [38]:
def df_add():
    new_df=pd.read_csv('기업 IPO 일정_수집.csv',encoding='utf-8',index_col=0)
    presen_df=pd.read_csv('기업 IPO 일정_업데이트.csv',encoding='utf-8')
    new_names=set(new_df['기업명'])-set(presen_df['기업명'])
    add_df=new_df[new_df['기업명'].isin(new_names)]
    added_df=pd.concat([add_df,presen_df])    
    
#     added_df.set_index('기업명',inplace=True)
#     presen_df.set_index('기업명',inplace=True)
#     added_df.update(presen_df)
    
    added_df.to_csv('기업 IPO 일정_업데이트.csv',encoding='utf-8')                      # 주피터 내부 저장용                
    
    desired=added_df['희망공모가(원)'].tolist()
    #     print(len(desired))
    #     print(len(result['기업명'].tolist()))
    low=[]
    high=[]
    for price in desired:
        price=str(price).strip().split('~')
    #         print(price)
        if len(price)==1:
            low.append(np.nan)
            high.append(np.nan)
        else:
            low.append(price[0])
            high.append(price[1])
    #     print(len(low),len(high))
    
    added_df.drop(['희망공모가(원)'],axis=1,inplace=True)
    added_df.insert(11,'희망공모가(하단)',low)
    added_df.insert(12,'희망공모가(상단)',high)

    
    d=time.strftime('%y%m%d',time.localtime(time.time()))
    
    
    added_df.to_csv(save_dir+f'기업 IPO 일정_{d}_{up}.csv',encoding='utf-8')          # 로컬 디렉토리 저장용

In [ ]:
df_add()

In [41]:
added_df.to_csv('기업 IPO 일정_업데이트_2.csv',encoding='utf-8')

In [39]:
desired=added_df['희망공모가(원)'].tolist()
#     print(len(desired))
#     print(len(result['기업명'].tolist()))
low=[]
high=[]
for price in desired:
    price=str(price).strip().split('~')
#         print(price)
    if len(price)==1:
        low.append(np.nan)
        high.append(np.nan)
    else:
        low.append(price[0])
        high.append(price[1])
#     print(len(low),len(high))

added_df.drop(['희망공모가(원)'],axis=1,inplace=True)
added_df.insert(11,'희망공모가(하단)',low)
added_df.insert(12,'희망공모가(상단)',high)

In [40]:
added_df.to_csv()

,청구일,승인일,자본금(백만),매출액(백만),당기순이익(백만),주업종,주간사,IR 일자,IR 장소/시간,수요예측일,...,공모주일정,청약경쟁률,분석링크,신규상장일,현재가(원),전일비(%),공모가대비등락률(%),시초가(원),시초/공모(%),첫날종가(원)
기업명,,,,,,,,,,,,,,,,,,,,,
이삭엔지니어링,2020/10/20,NaN,"3,062","32,759","2,679","컴퓨터 프로그래밍, 시스템 통합",신한금융투자,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
아모센스,2020/10/19,NaN,"4,447","62,028","-1,196",전자부품 제조업,신한금융투자,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
유안타스팩7호,2020/10/16,NaN,115,0,0,금융 지원 서비스업,유안타증권,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
유안타스팩8호,2020/10/16,NaN,24,0,0,금융 지원 서비스업,유안타증권,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
디비금융스팩9호,2020/10/15,NaN,84,0,0,기타 금융업,DB금융투자,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
다날,NaN,NaN,NaN,NaN,NaN,NaN,LG증권,NaN,NaN,NaN,...,2004.07.13~07.14,68.25 : 1,http://www.38.co.kr/html/fund/index.htm?o=v&no...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
텔코웨어(유가),NaN,NaN,NaN,NaN,NaN,NaN,삼성증권,NaN,NaN,NaN,...,2004.07.08~07.09,45.72 : 1,http://www.38.co.kr/html/fund/index.htm?o=v&no...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
아이레보,NaN,NaN,NaN,NaN,NaN,NaN,교보증권,NaN,NaN,NaN,...,2003.12.15~12.16,NaN,http://www.38.co.kr/html/fund/index.htm?o=v&no...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
